In [1]:
from glob import glob
import os
from segment_anything.build_sam import sam_model_registry
from scripts.experiments.mask_aug.inference import class_inference, load_model
# from scripts.sam_train import SamTrain
# from segment_anything.modeling.sam import Sam
from scripts.datasets.constant import IMAGE_TYPE
from scripts.datasets.preprocess_raw import FLARE22_Preprocess

TRAIN_ROOT = "../dataset/FLARE22-version1/FLARE22_LabeledCase50"
TRAIN_IMAGE_PATH = os.path.join(TRAIN_ROOT, "images/*.nii.gz")
TRAIN_MASK_PATH = os.path.join(TRAIN_ROOT, "labels/*.nii.gz")

VAL_ROOT = "../dataset/FLARE22-version1/ReleaseValGT-20cases"
VAL_IMAGE_PATH = os.path.join(VAL_ROOT, "images/*.nii.gz")
VAL_MASK_PATH = os.path.join(VAL_ROOT, "labels/*.nii.gz")

train_images = sorted(list(glob(TRAIN_IMAGE_PATH)))
train_masks = sorted(list(glob(TRAIN_MASK_PATH)))
train_file = list(zip(train_images, train_masks))

val_images = sorted(list(glob(VAL_IMAGE_PATH)))
val_masks = sorted(list(glob(VAL_MASK_PATH)))
val_file = list(zip(val_images, val_masks))

# MODEL_PATH = "../runs/transfer/imp-230603-150046/model-20.pt"
# MODEL_PATH = "../runs/exps-230701-165310/model-20.pt"

# model: Sam = sam_model_registry["vit_b"](
#         checkpoint="../sam_vit_b_01ec64.pth", custom=MODEL_PATH
#     )
# sam_train = SamTrain(sam_model=model)


In [2]:
import torch
from tqdm import tqdm
from scripts.datasets.constant import FLARE22_LABEL_ENUM
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

preprocessor = FLARE22_Preprocess()


In [ ]:
from scripts.experiments.ssm.dataset import MeanShapeDataset
MeanShapeDataset.TRAIN_CACHE_NAME = "../mean-shape/train/"
MeanShapeDataset.VAL_CACHE_NAME = "../mean-shape/validation/"
dataset = MeanShapeDataset(is_train_set=True)

In [57]:
print(dataset.mean_z)
volumes, masks = preprocessor.run_with_config(
    image_file=train_file[0][0],
    gt_file=train_file[0][1],
    config_name=IMAGE_TYPE.ABDOMEN_SOFT_TISSUES_ABDOMEN_LIVER,
)
volumes = volumes[::-1]
masks = masks[::-1]

95.88


In [63]:
from scripts.experiments.ssm.model import SimpleSDFModel

model = SimpleSDFModel(n_input=3, n_output=1, n_hidden=10)
model.load_state_dict(torch.load("../runs/stat-shape-230706-005124/model-8.pt"))
with torch.no_grad():
    coords = np.argwhere(np.ones(volumes.shape))
    normalized_coords = coords / np.array([[dataset.mean_z, 512, 512]])
    pred = model(torch.Tensor(normalized_coords))
    pred = pred.numpy()
    mi, ma, me, st = np.min(pred), np.max(pred), np.mean(pred), np.std(pred)
    print(f"Statistic: {mi:.3f}, {ma:.3f}, {me:.3f}, {st:.3f}")

Statistic: 0.002, 0.300, 0.042, 0.045


In [47]:
merge_arr = np.pad(coords, pad_width=((0, 0), (0, 1)), constant_values=0.0)
merge_arr[:, 3:] = pred


True

False

In [50]:
merge_arr[:, 3] == True

False